In [98]:
#%pip install pandas-datareader
%pip install yfinance
#!pip install yfinance --upgrade --no-cache-dir

Note: you may need to restart the kernel to use updated packages.


In [99]:
import pandas as pd
from datetime import datetime, timedelta
import yfinance as yf

In [100]:
pd.set_option('display.float_format', lambda x: '%.8f' % x) 
# change the visualization of a float to eight decimal digits. Change this to a with statement.

In [101]:
binance = pd.read_csv('data/final/saldo_binance.csv', index_col=0) # final (it means wallets.py needs to run first)
trezor = pd.read_csv('data/final/saldo_trezor.csv', index_col=0)
binance_complete = pd.read_csv('data/final/binance_dates.csv', index_col=0)

In [102]:
tickers = list(set(binance.index).union(trezor.index).union(binance_complete.index))
tickers = ['ETH', 'MATIC', 'BTC'] # selecting only the relevant ones. (can change)

In [103]:
# Set the start and end dates for the historical data (this one is for today price)
start_date = (datetime.now()- timedelta(days=1)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

# Fetch the historical data for each ticker using Yahoo Finance API
hist_data = {}
for ticker in tickers:
    data = yf.download(ticker + '-USD', start=start_date, end=end_date)
    hist_data[ticker] = data['Close']

# Combine the historical data for all tickers into a single dataframe
df = pd.concat(hist_data, axis=1, keys=tickers)

# Print the head of the combined dataframe
print(df.head())


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
                     ETH      MATIC            BTC
Date                                              
2023-11-20 2022.23913574 0.81151801 37476.95703125


In [104]:
# Merge the two DataFrames on the 'Coin' column
data = pd.merge(binance, trezor, left_index=True, right_index=True, how='outer')
# summing values
data['Balance'] = data['Amount'].add(data['Balance'], fill_value=0)
data['BRL_spent'] = data['BRL_spent_x'].add(data['BRL_spent_y'], fill_value=0)
# Drop unnecessary columns
data = data.drop(['BRL_spent_x', 'Mean_price_x', 'Amount',
       'BRL_spent_y', 'Mean_price_y'], axis=1)
# calculate mean price
data['Mean_price'] = data['BRL_spent'] / data['Balance'] 

In [105]:
# Corretagem do dolar
# Download the data
dolar_hoje = yf.download('BRLUSD=X', start=start_date, end=end_date)

# Print the last available exchange rate
if not data.empty:
    last_exchange_rate = 1/dolar_hoje['Close'].iloc[-1]
    print(f'The last exchange rate for BRL to USD is: {last_exchange_rate}')
else:
    print('No data available.')

[*********************100%***********************]  1 of 1 completed
The last exchange rate for BRL to USD is: 4.819177188583319


In [108]:
# Concatenate the two DataFrames based on the common index values
result_df = pd.concat([data.loc[['BTC', 'ETH', 'MATIC']], df.T * last_exchange_rate], axis=1)
result_df['current_value'] = result_df.iloc[:,-1] * result_df['Balance']
result_df['lucro'] = result_df['current_value'] - result_df['BRL_spent']
result_df['lucro%'] = result_df['lucro'] / result_df['BRL_spent'] * 100
result_df

,UTC_Time,Balance,BRL_spent,Mean_price,2023-11-20 00:00:00,current_value,lucro,lucro%
BTC,2022-01-04 19:50:53,0.22548431,28056.75570591,124428.86028705,180608.09642252,40724.29200224,12667.53629633,45.14968312
ETH,2022-01-08 22:46:43,0.26100225,2339.89590157,8965.04111199,9745.52871283,2543.60492149,203.70901992,8.70590097
MATIC,2022-08-24 14:43:07,122.13416286,399.00000000,3.26689921,3.91084910,477.64828074,78.64828074,19.71134856


In [112]:
result_df[['Mean_price', 'current_value','lucro%']]

,Mean_price,current_value,lucro%
BTC,124428.86028705,40724.29200224,45.14968312
ETH,8965.04111199,2543.60492149,8.70590097
MATIC,3.26689921,477.64828074,19.71134856
